<a href="https://colab.research.google.com/github/ViniciusCastillo/BootcampAlura_ProjetoModulo3/blob/main/notebooks/Funcoes_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação de Funções e Dados Auxiliares

Neste notebook temos as funções que serão utilizadas no notebook de análise e projeção, bem como alguns dados auxiliares

## Importanto bibliotecas que serão utilizadas

In [381]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
# configurando formato de apresentação de números
pd.options.display.float_format = "{:,.2f}".format

## **Funções**

### Função <font color='blue'>metricas</font><font size=3>(df_real, df_previsto, titulo)</font>
####Cálcula as métricas de comparação entre o dataframe real e o previsto

##### **Entradas e Saídas**

Informações de entrada:
* <font color='green'>df_real</font>: data frame real
* <font color='green'>df_previsto</font>: data frame previsto
* <font color='green'>titulo</font>: titulo da comparação que será realizada

Saídas:
* Printa o <font color='green'>título</font> da comparação, depois o resultado de 3 métricas:
> 1. **MAE**: Mean Absolute Error
> 1. **RMSE**: Root Mean Squared Error
> 1. **MAPE**: Mean Absolute Percentage Error

* Está função não retorna nenhum valor

In [382]:
def metricas (df_real, df_previsto, titulo):
  print ("\n"+ titulo)
  print ("Mean Absolute Error: %.2f" % mean_absolute_error(df_real['y'], df_previsto['yhat']))
  print ("Root Mean Squared Error: {:,.2f}".format(mean_squared_error(df_real['y'], df_previsto['yhat'])))
  print ("Mean Absolute Percentage Error: {:,.2f}".format(mean_absolute_percentage_error(df_real['y'], df_previsto['yhat'])))

### Função <font color='blue'>cria_DFs</font><font size=3>(dados, campo_y, inicio_teste, inicio_treino=<font color='yellow'>0</font>, campo_data=<font color='yellow'>'date'</font>)</font>
#### Divide uma base da dados entre uma base de treino e outra base de teste para criação de modelos preditivos, já no padrão do Facebook Prophet

##### **Entradas e Saídas**

Informações de entrada:
* <font color='green'>dados</font>: data frame que será dividido
* <font color='green'>campo_y</font>: campo do data frame que será utilizado como a varável ***'y'*** do modelo preditivo do Prophet
* <font color='green'>inicio_teste</font>: valor do registro inicial para a base de teste, consequentemente será utilizado como o fim da base de treino
* <font color='green'>inicio_treino</font>: valor do registro inicial para a base de treino, por padrão tem o valor <font color='yellow'>0</font>
* <font color='green'>campo_data</font>: campo do data frame que possuí a data e que será utilizado na varável ***'ds'*** do modelo preditivo do Prophet, por padrão tem o valor <font color='yellow'>'date'</font>

Saídas:
* Esta função retorna 2 informações:
> 1. <font color='purple'>df_treino</font>: data frame de treino
> 1. <font color='purple'>df_teste</font>: data fram de teste
* Estes data frames retornados estão com o seu index resetado
* Precisa ser atribuida para uma dupla de variáveis, por exemplo: *df_treino, df_teste = cria_DFs(dados, campo_y, inicio_teste)*

In [383]:
def cria_DFs (dados, campo_y, inicio_teste, inicio_treino=0, campo_data='date'):
  df_teste = pd.DataFrame()
  # a base de teste será sempre do início enviado até o final da base de dados
  df_teste['ds'] = dados[campo_data][inicio_teste:]
  df_teste['y'] = dados[campo_y][inicio_teste:]
  df_teste = df_teste.reset_index(drop=True)
  df_treino = pd.DataFrame()
  # a base de treino terá como fim o início do teste enviado
  df_treino['ds'] = dados[campo_data][inicio_treino:inicio_teste]
  df_treino['y'] = dados[campo_y][inicio_treino:inicio_teste]
  df_treino = df_treino.reset_index(drop=True)
  return df_treino, df_teste

### Função <font color='blue'>modelar</font><font size=3>(df_treino, df_teste, y_label, show_change_points=<font color='yellow'>False</font>, **todos os parâmetros da função Prophet da biblioteca Facebook Prophet)</font>
#### Cria o modelo preditivo da biblioteca Facebook Prophet, retorna o modelo e a base de previsão. Fora isso, exibe o gráfico do modelo, incluindo a comparação com a base de teste

##### **Entradas e Saídas**

Informações de entrada:
* <font color='green'>df_treino</font>: data frame que será utilizado para trienar o modelo. **Obersevação**: lembrar que o se o parâmetro *growth* for alterado para *'logistic'* é necessário incluir o campo *'cap'<font size=1><sup>(1)</sup></font>* neste data frame antes de chamar esta função
* <font color='green'>df_teste</font>: data frame que será plotado como comparação para a projeção
* <font color='green'>y_label</font>: valor que será plotado como título do eixo Y do gráfico
* <font color='green'>show_change_points</font>: se passar o valor <font color='grey'>**True**</font> plota os change points no gráfico, por padrão o valor é <font color='yellow'>False</font>
* <font color='green'>** parâmetros do Prophet</font>: pode-se passar qualquer parâmentro da função Prophet da biblioteca Facebook Prophet

<font size=2> <font size=1><sup>(1)</sup></font> *'cap' é a definição do valor máximo esperado na função logística e precisa ter valor em todas as linhas do data frame. [Link para página do Prophet sobre isso](https://facebook.github.io/prophet/docs/saturating_forecasts.html#forecasting-growth)*</font>

Saídas:
* Esta função retorna 2 informações:
> 1. <font color='purple'>modelo</font>: o modelo calculado pelo Prophet
> 1. <font color='purple'>previsao</font>: a base de previsão calculada pelo Prophet. Vale destacar que os dados futuros terão o mesmo tamanho que o <font color='green'>df_teste</font> que foi passado
* Ela também plota o gráfico do modelo previsto (<font color='purple'>previsao</font> como uma linha azul), bem como as informações reais, tanto do <font color='green'>df_treino</font> (linha cinza) quanto <font color='green'>df_teste</font> (linha laranja), para comparação visual
* Caso tenha optado pela opção de mostrar os change points, eles irão aparecer no gráfico também

In [384]:
def modelar (df_treino, df_teste, y_label, show_change_points=False, growth='linear', changepoints=None, n_changepoints=25, changepoint_range=0.8, 
             yearly_seasonality='auto', weekly_seasonality='auto', daily_seasonality='auto', holidays=None, seasonality_mode='additive', 
             seasonality_prior_scale=10.0, holidays_prior_scale=10.0, changepoint_prior_scale=0.05, mcmc_samples=0, interval_width=0.8, 
             uncertainty_samples=1000, stan_backend=None):
  # cria o modelo com os parâmetros escolhidos
  modelo = Prophet(growth, changepoints, n_changepoints, changepoint_range, yearly_seasonality, weekly_seasonality, daily_seasonality, 
                   holidays, seasonality_mode, seasonality_prior_scale, holidays_prior_scale, changepoint_prior_scale, mcmc_samples, 
                   interval_width, uncertainty_samples, stan_backend)
  # treina o modelo com o df_treino
  modelo.fit(df_treino)
  # cria o df para previsão futura do mesmo tamanho que o df_teste
  df_futuro = modelo.make_future_dataframe(periods=len(df_teste))
  # se estiver projetando um modelo logístico, inclui o 'cap' no df_futuro
  if growth=='logistic': df_futuro['cap'] = df_treino['cap'].iloc[0];
  # preve os pontos futuros com base no modelo treinado anteriormente
  previsao = modelo.predict(df_futuro)
  # plota a previsão, onde o nome do eixo y é igual ao que foi passado na função
  fig = modelo.plot(previsao, xlabel='Data', ylabel=y_label, figsize=(15,8))
  # se estiver definido show_change_points como True, plota os change points
  if show_change_points: a = add_changepoints_to_plot(fig.gca(), modelo, previsao);
  # plota o df_teste como uma linha pontilhada laranja
  plt.plot(df_teste.ds, df_teste.y, ':', marker='.', color='orange')
  # plota o df_treino como uma linha pontilhada cinza
  plt.plot(df_treino.ds, df_treino.y, ':', color="grey")
  # plt.xticks(color='black')
  # plt.yticks(color='black')
  return modelo, previsao # retorna as informações do modelo que foi treinado e o df da previsão criada

### Função <font color='blue'>agrupa_EW</font><font size=3>(df, campo_data=<font color='yellow'>'date'</font>, campo_EW=<font color='yellow'>'epidemiological_week'</font>, campos=<font color='yellow'>[ ]</font>)</font>
#### Retorna um data frame agrupado por semana epidemiológica, com a data do primeiro dia da semana e somando os valores dos campos numéricos restantes

##### **Entradas e Saídas**

Informações de entrada:
* <font color='green'>df</font>: data frame que será agrupado
* <font color='green'>campo_data</font>: campo de data do data frame enviado, por padrão será utilizado <font color='yellow'>'date'</font>
* <font color='green'>campo_EW</font>: campo que possuí a semana epidemiológica, por padrão será utilizado <font color='yellow'>'epidemiological_week'</font>
* <font color='green'>campos</font>: lista dos demais campos não numéricos para manter na tabela, por padrão é uma <font color='yellow'>lista vazia</font>

Saídas:
* Esta função retorna o data frame <font color='purple'>df_saida</font> com as seguintes alterações:
> 1. <font color='green'>campo_data</font>: passará a ser data do primeiro dia da semana epidemiológica
> 1. <font color='green'>campo_EW</font>: será excluído
> 1. Os demais <font color='green'>campos</font> não numéricos que foram passados na função
> 1. A soma dos campos numéricos da base original


In [385]:
def agrupa_EW (df, campo_data='date', campo_EW='epidemiological_week', campos=[]):
  df_saida = df.copy()
  df_saida[campo_data]=df_saida[campo_EW].map(EW_Date)
  df_saida.drop(columns=campo_EW, inplace=True)
  lista = []
  lista += [campo_data]
  lista += campos
  df_saida = df_saida.groupby(by=lista).sum().reset_index()
  return df_saida.reset_index(drop=True)

### Função <font color='blue'>valid_cruzada</font><font size=3>(modelo, inicial=<font color='yellow'>'180 days'</font>, periodo=<font color='yellow'>'30 days'</font>, horizonte=<font color='yellow'>'30 days'</font>, metrica=<font color='yellow'>'rmse'</font>)</font>
#### Retorna um data frame da validação cruzada e também plota um gráfico com a métrica escolhida, por padrão é a métrica do Root Mean Squared Error

##### **Entradas e Saídas**

Informações de entrada:
* <font color='green'>modelo</font>: modelo do Prophet para validação
* <font color='green'>inicial</font>: período inicial para a primeira validação, por padrão será utilizado <font color='yellow'>'180 days'</font>
* <font color='green'>periodo</font>: janela de tempo para cada nova validação, por padrão será utilizado <font color='yellow'>'30 days'</font>
* <font color='green'>horizonte</font>: janela de projeção de cada validação, por padrão será utilizado <font color='yellow'>'30 days'</font>
* <font color='green'>metrica</font>: métrica que será utilizada para plotar o gráfico, por padrão será utilizado <font color='yellow'>'rmse'</font>

Saídas:
* Esta função retorna o data frame <font color='purple'>df_cv</font> da validação cruzada
* Ela também irá plotar um gráfico da validação cruzada com a métrica escolhida

In [386]:
def valid_cruzada(modelo, inicial='180 days', periodo='30 days', horizonte='30 days', metrica='rmse'):
  df_cv = cross_validation(modelo, initial=inicial, period=periodo, horizon=horizonte)
  fig = plot_cross_validation_metric(df_cv ,metric='rmse')
  return df_cv

## **Dados**

### lista de feriados da cidade de São Paulo: <font color=blue>feriados_sp</font>

#### lista de feriados da cidade de São Paulo de 2020

In [387]:
feriados_sp_2020 = [
'2020-02-25',
'2020-02-26',
'2020-04-10',
'2020-02-25',
'2020-02-26',
'2020-04-10',
'2020-04-21',
'2020-05-01',
'2020-05-20',
'2020-05-21', 
'2020-05-22',
'2020-05-25',
'2020-06-11',
'2020-09-07',
'2020-10-12',
'2020-11-02',
'2020-11-15',
'2020-12-25'
]

In [388]:
feriados_sp_2020 = pd.DataFrame({
  'holiday': 'sp_2020',
  'ds': pd.to_datetime(feriados_sp_2020)
})

#### lista de feriados da cidade de São Paulo de 2021

In [389]:
feriados_sp_2021 = [
'2021-01-01',
'2021-01-25',
'2021-02-15',
'2021-02-16',
'2021-02-17',
'2021-03-26',
'2021-03-29',
'2021-03-30',
'2021-03-31',
'2021-04-01',
'2021-04-02',
'2021-04-21',
'2021-05-01',
'2021-06-03',
'2021-07-09',
'2021-09-07',
'2021-10-12',
'2021-11-02',
'2021-11-15',
'2021-12-25'
]

In [390]:
feriados_sp_2021 = pd.DataFrame({
  'holiday': 'sp_2021',
  'ds': pd.to_datetime(feriados_sp_2021)
})


#### lista consolida de feriados da cidade de São Paulo 2020 e 2021

In [391]:
feriados_sp = pd.concat((feriados_sp_2020, feriados_sp_2021))
print(feriados_sp.head())
print(feriados_sp.tail())

   holiday         ds
0  sp_2020 2020-02-25
1  sp_2020 2020-02-26
2  sp_2020 2020-04-10
3  sp_2020 2020-02-25
4  sp_2020 2020-02-26
    holiday         ds
15  sp_2021 2021-09-07
16  sp_2021 2021-10-12
17  sp_2021 2021-11-02
18  sp_2021 2021-11-15
19  sp_2021 2021-12-25


### Dicionários de estados

#### Estados e siglas: <font color='blue'>ufs_siglas</font> e <font color='blue'>siglas_ufs</font>

In [392]:
ufs_siglas = {"Acre":"AC","Alagoas":"AL","Amapá":"AP","Amazonas":"AM","Bahia":"BA","Ceará":"CE",
           "Distrito Federal":"DF","Espírito Santo":"ES","Goiás":"GO","Maranhão":"MA","Mato Grosso":"MT",
           "Mato Grosso do Sul":"MS","Minas Gerais":"MG","Pará":"PA","Paraíba":"PB","Paraná":"PR",
           "Pernambuco":"PE","Piauí":"PI","Rio de Janeiro":"RJ","Rio Grande do Norte":"RN","Rio Grande do Sul":"RS",
           "Rondônia":"RO","Roraima":"RR","Santa Catarina":"SC","São Paulo":"SP","Sergipe":"SE","Tocantins":"TO"}

In [393]:
# código para inverter a ordem do dicionário anterior
siglas_ufs = {}
for uf, sigla in ufs_siglas.items(): 
  siglas_ufs[sigla] = uf

#### Estados e Regiões: <font color='blue'>ufs_regioes</font>

In [394]:
ufs_regioes = {"Acre":"Norte","Alagoas":"Nordeste","Amapá":"Norte","Amazonas":"Norte","Bahia":"Nordeste",
               "Ceará":"Nordeste","Distrito Federal":"Centro-Oeste","Espírito Santo":"Sudeste","Goiás":"Centro-Oeste",
               "Maranhão":"Nordeste","Mato Grosso":"Centro-Oeste","Mato Grosso do Sul":"Centro-Oeste",
               "Minas Gerais":"Sudeste","Pará":"Norte","Paraíba":"Nordeste","Paraná":"Sul","Pernambuco":"Nordeste",
               "Piauí":"Nordeste","Rio de Janeiro":"Sudeste","Rio Grande do Norte":"Nordeste",
               "Rio Grande do Sul":"Sudeste","Rondônia":"Norte","Roraima":"Norte","Santa Catarina":"Sul",
               "São Paulo":"Sudeste","Sergipe":"Nordeste","Tocantins":"Norte"}

### Dados Brasil.io

#### importando os dados

Os dados tiveram um tratamento inicial no notebook [TratandoBaseOriginal](https://github.com/ViniciusCastillo/BootcampAlura_ProjetoModulo3/blob/main/notebooks/TratandoBaseOriginal.ipynb) para reduzir o seu tamanho e permitir a carga no GitHub.

Foram mantidas apenas as maiores cidades do Brasil e a visão consolidada dos estados.

In [395]:
dados = pd.read_csv('https://raw.githubusercontent.com/ViniciusCastillo/BootcampAlura_ProjetoModulo3/main/dados/dados_covid.csv')
dados['date'] = pd.to_datetime(dados['date'])
dados['UF'] = dados['state'].map(siglas_ufs)
dados['regiao'] = dados['UF'].map(ufs_regioes)
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23267 entries, 0 to 23266
Data columns (total 20 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   city                                           6307 non-null   object        
 1   city_ibge_code                                 23267 non-null  float64       
 2   date                                           23267 non-null  datetime64[ns]
 3   epidemiological_week                           23267 non-null  int64         
 4   estimated_population                           23267 non-null  float64       
 5   estimated_population_2019                      23267 non-null  float64       
 6   is_last                                        23267 non-null  bool          
 7   is_repeated                                    23267 non-null  bool          
 8   last_available_confirmed                       23267 non

#### base das cidades mais populosas: <font color='blue'>dados_cidades</font>
formato:

|Campo|Descrição
|:----|:----|
|**city**|Cidade|
|**UF**|Unidade da federação|
|**regiao**|Região do Brasil que a unidade da federação pertence|
|**date**|Data: Ano-Mês-Dia (datetime)|
|**epidemiological_week**|Semana epidemológica: AnoSemana (int)|
|**estimated_population**|População estimadade de 2020 (float)|
|**new_cofirmed**|Novos casos (int)|
|**new_deaths**|Novos óbitos (int)|

In [396]:
dados_cidades = dados.query('place_type == "city"')[['city','UF','regiao','date','epidemiological_week','estimated_population','new_confirmed','new_deaths']]
dados_cidades.reset_index(inplace=True, drop=True)
dados_cidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6307 entries, 0 to 6306
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   city                  6307 non-null   object        
 1   UF                    6307 non-null   object        
 2   regiao                6307 non-null   object        
 3   date                  6307 non-null   datetime64[ns]
 4   epidemiological_week  6307 non-null   int64         
 5   estimated_population  6307 non-null   float64       
 6   new_confirmed         6307 non-null   int64         
 7   new_deaths            6307 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 394.3+ KB


#### base de estados : <font color='blue'>dados_estados</font>
formato:

|Campo|Descrição
|:----|:----|
|**UF**|Unidade da federação|
|**regiao**|Região do Brasil que a unidade da federação pertence|
|**date**|Data: Ano-Mês-Dia (datetime)|
|**epidemiological_week**|Semana epidemológica: AnoSemana (int)|
|**estimated_population**|População estimadade de 2020 (float)|
|**new_cofirmed**|Novos casos (int)|
|**new_deaths**|Novos óbitos (int)|

In [397]:
dados_estados = dados.query('place_type == "state"')[['UF', 'regiao','date','epidemiological_week','estimated_population','new_confirmed','new_deaths']]
dados_estados.reset_index(inplace=True, drop=True)
dados_estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16960 entries, 0 to 16959
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   UF                    16960 non-null  object        
 1   regiao                16960 non-null  object        
 2   date                  16960 non-null  datetime64[ns]
 3   epidemiological_week  16960 non-null  int64         
 4   estimated_population  16960 non-null  float64       
 5   new_confirmed         16960 non-null  int64         
 6   new_deaths            16960 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 927.6+ KB


#### base de regiões : <font color='blue'>dados_regioes</font>
formato:

|Campo|Descrição
|:----|:----|
|**regiao**|Região do Brasil que a unidade da federação pertence|
|**date**|Data: Ano-Mês-Dia (datetime)|
|**epidemiological_week**|Semana epidemológica: AnoSemana (int)|
|**estimated_population**|População estimadade de 2020 (float)|
|**new_cofirmed**|Novos casos (int)|
|**new_deaths**|Novos óbitos (int)|

In [398]:
dados_regioes = dados_estados.groupby(by=['regiao','date','epidemiological_week']).sum().reset_index()
dados_regioes.reset_index(inplace=True, drop=True)

In [399]:
# ajustando a população estimada dos primeiros meses do ano, partindo do último mês como referência
regiao_populacao_estimada = dados_regioes.query("date == '2021-11-30'")[['regiao','estimated_population']].set_index('regiao')
dados_regioes.drop(columns='estimated_population', inplace=True)
dados_regioes = dados_regioes.join(regiao_populacao_estimada, how='left', on='regiao')
dados_regioes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3171 entries, 0 to 3170
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   regiao                3171 non-null   object        
 1   date                  3171 non-null   datetime64[ns]
 2   epidemiological_week  3171 non-null   int64         
 3   new_confirmed         3171 non-null   int64         
 4   new_deaths            3171 non-null   int64         
 5   estimated_population  3171 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 148.8+ KB


#### base total do Brasil : <font color='blue'>dados_Brasil</font>
formato:

|Campo|Descrição
|:----|:----|
|**date**|Data: Ano-Mês-Dia (datetime)|
|**epidemiological_week**|Semana epidemológica: AnoSemana (int)|
|**estimated_population**|População estimadade de 2020 (float)|
|**new_cofirmed**|Novos casos (int)|
|**new_deaths**|Novos óbitos (int)|

In [400]:
dados_Brasil = dados_regioes.groupby(by=['date','epidemiological_week']).sum().reset_index()
dados_Brasil.reset_index(inplace=True, drop=True)

In [401]:
# ajustando a população estimada dos primeiros meses do ano, partindo do último mês como referência
populacao_estimada = dados_Brasil.query("date == '2021-11-30'")[['estimated_population']]
dados_Brasil['estimated_population'] = float (populacao_estimada['estimated_population'])
print (dados_Brasil.info())
dados_Brasil.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  645 non-null    datetime64[ns]
 1   epidemiological_week  645 non-null    int64         
 2   new_confirmed         645 non-null    int64         
 3   new_deaths            645 non-null    int64         
 4   estimated_population  645 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 25.3 KB
None


,date,epidemiological_week,new_confirmed,new_deaths,estimated_population
0,2020-02-25,202009,1,0,"211,755,692.00"
1,2020-02-26,202009,0,0,"211,755,692.00"
2,2020-02-27,202009,0,0,"211,755,692.00"
3,2020-02-28,202009,1,0,"211,755,692.00"
4,2020-02-29,202009,0,0,"211,755,692.00"


### Epidemiological Week Date: <font color='blue'>EW_Date</fonte>
Dicionário com uma data para cada semana epidemiológica

In [402]:
epidemiological_week = dados_Brasil[['epidemiological_week','date']].groupby('epidemiological_week').min()
EW_Date = {}
for item in epidemiological_week.index:
  EW_Date[item] = epidemiological_week['date'].loc[item]
EW_Date

{202009: Timestamp('2020-02-25 00:00:00'),
 202010: Timestamp('2020-03-01 00:00:00'),
 202011: Timestamp('2020-03-08 00:00:00'),
 202012: Timestamp('2020-03-15 00:00:00'),
 202013: Timestamp('2020-03-22 00:00:00'),
 202014: Timestamp('2020-03-29 00:00:00'),
 202015: Timestamp('2020-04-05 00:00:00'),
 202016: Timestamp('2020-04-12 00:00:00'),
 202017: Timestamp('2020-04-19 00:00:00'),
 202018: Timestamp('2020-04-26 00:00:00'),
 202019: Timestamp('2020-05-03 00:00:00'),
 202020: Timestamp('2020-05-10 00:00:00'),
 202021: Timestamp('2020-05-17 00:00:00'),
 202022: Timestamp('2020-05-24 00:00:00'),
 202023: Timestamp('2020-05-31 00:00:00'),
 202024: Timestamp('2020-06-07 00:00:00'),
 202025: Timestamp('2020-06-14 00:00:00'),
 202026: Timestamp('2020-06-21 00:00:00'),
 202027: Timestamp('2020-06-28 00:00:00'),
 202028: Timestamp('2020-07-05 00:00:00'),
 202029: Timestamp('2020-07-12 00:00:00'),
 202030: Timestamp('2020-07-19 00:00:00'),
 202031: Timestamp('2020-07-26 00:00:00'),
 202032: Ti